# Programming language interoperability (Interop)

## Python

In [2]:
using PythonCall

In [3]:
@pyeval "3+3"

Python int: 6

In [4]:
np = pyimport("numpy")

Python module: <module 'numpy' from '/Users/crstnbr/repos/JuliaWorkshops/JuliaHLRS22/.CondaPkg/env/lib/python3.10/site-packages/numpy/__init__.py'>

In [5]:
np.linalg.eigvals(np.random.rand(5,5))

Python ndarray:
array([ 2.60900561+0.j        , -0.45160431+0.46032981j,
       -0.45160431-0.46032981j,  0.41543947+0.09148292j,
        0.41543947-0.09148292j])

In [6]:
M = rand(5,5)
np.linalg.eigvals(M)

Python ndarray:
array([ 2.54126959+0.j        ,  0.27103712+0.27802955j,
        0.27103712-0.27802955j, -0.15730008+0.27536292j,
       -0.15730008-0.27536292j])

In [7]:
@pyexec """
global sinpi, np
import numpy as np

def sinpi(x):
    return np.sin(np.pi * x)
"""

In [8]:
py_sinpi(x) = pyconvert(Float64, @pyeval("sinpi")(x))

py_sinpi (generic function with 1 method)

In [9]:
py_sinpi(10)

-1.2246467991473533e-15

In [10]:
using BenchmarkTools
@btime py_sinpi(10);
@btime sinpi(10); # built-in Julia function

  1.795 μs (3 allocations: 48 bytes)
  1.984 ns (0 allocations: 0 bytes)


## C

In [13]:
c_code = """
#include <stddef.h>
double c_sum(size_t n, double *X) {
    double s = 0.0;
    for (size_t i = 0; i < n; ++i) {
        s += X[i];
    }
    return s;
}
""";

Compile to a shared library by piping `c_code` to gcc:

In [14]:
using Libdl
const Clib = tempname() * "." * Libdl.dlext

open(`gcc -fPIC -O3 -msse3 -xc -shared -o $Clib -`, "w") do f
    print(f, c_code)
end

In [15]:
Clib

"/var/folders/1d/sbf5s24x6y306tgv_4hh06y00000gn/T/jl_hezSxybUR8.dylib"

Binding the function from the shared library:

In [16]:
c_sum(X::Array{Float64}) = @ccall Clib.c_sum(length(X)::Csize_t, X::Ptr{Float64})::Float64

c_sum (generic function with 1 method)

In [17]:
c_sum(rand(10))

4.112260824926674

In [18]:
x = rand(10)
@btime c_sum($x);

  8.095 ns (0 allocations: 0 bytes)


## Mixing Julia, Python, and C

Julia (`real`), Python/numpy (`py_sinpi`), C (`c_sum`)

In [19]:
x = rand(10);

In [21]:
abs(py_sinpi(c_sum(x)))

0.7668959571492515

In [22]:
@btime abs(py_sinpi(c_sum($x)));

  1.805 μs (3 allocations: 48 bytes)


See [JuliaInterop](https://github.com/JuliaInterop) for more, such as [RCall.jl](https://github.com/JuliaInterop/RCall.jl), [JavaCall.jl](https://github.com/JuliaInterop/JavaCall.jl), and [MATLAB.jl](https://github.com/JuliaInterop/MATLAB.jl).

# Julia Microbenchmark: Summation

Let's look at and benchmark the sum function:

$$\mathrm{sum}(x) = \sum_{i=1}^n x_i$$

In [23]:
x = rand(10^7);

In [24]:
sum(x)

4.999245210168353e6

In [25]:
d = Dict() # to store the measurement results

Dict{Any, Any}()

## Python

In [26]:
using BenchmarkTools
using PythonCall

### numpy

In [27]:
np = pyimport("numpy")

Python module: <module 'numpy' from '/Users/crstnbr/repos/JuliaWorkshops/JuliaHLRS22/.CondaPkg/env/lib/python3.10/site-packages/numpy/__init__.py'>

In [28]:
numpy_sum = np.sum

Python function: <function sum at 0x11d765ab0>

In [29]:
b = @benchmark $numpy_sum($x)

BenchmarkTools.Trial: 790 samples with 1 evaluation.
 Range (min … max):  6.077 ms …   9.187 ms  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     6.168 ms               ┊ GC (median):    0.00%
 Time  (mean ± σ):   6.321 ms ± 387.274 μs  ┊ GC (mean ± σ):  0.00% ± 0.00%

  ██▇▅▃▃▂▂▂▁▃▁▂▂▁                                              
  ████████████████▇▆▅▆▆▆▄▇▄▆▆▅▆▆▁▁▄▆▅▅▄▁▄▅▁▁▅▄▁▁▅▁▁▁▄▁▁▁▁▁▁▄▄ █
  6.08 ms      Histogram: log(frequency) by time      8.21 ms <

 Memory estimate: 928 bytes, allocs estimate: 23.

In [30]:
d["Python (numpy)"] = minimum(b.times) / 1e6

6.077495

### hand-written

In [31]:
@pyexec """
global mysum

def mysum(a):
    s = 0.0
    for x in a:
        s = s + x
    return s
"""

In [32]:
mysum_py = @pyeval("mysum")

Python function: <function mysum at 0x155e716c0>

In [33]:
x_py = pylist(x);

In [34]:
b = @benchmark $mysum_py($x_py)

BenchmarkTools.Trial: 15 samples with 1 evaluation.
 Range (min … max):  340.146 ms … 349.350 ms  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     343.763 ms               ┊ GC (median):    0.00%
 Time  (mean ± σ):   344.092 ms ±   2.536 ms  ┊ GC (mean ± σ):  0.00% ± 0.00%

  ▁          █  ▁▁▁   ▁  ▁ █    ▁        █               ▁    ▁  
  █▁▁▁▁▁▁▁▁▁▁█▁▁███▁▁▁█▁▁█▁█▁▁▁▁█▁▁▁▁▁▁▁▁█▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁█▁▁▁▁█ ▁
  340 ms           Histogram: frequency by time          349 ms <

 Memory estimate: 16 bytes, allocs estimate: 1.

In [35]:
d["Python (hand-written)"] = minimum(b.times) / 1e6

340.145828

### built-in

In [37]:
# get the Python built-in "sum" function:
pysum = pybuiltins.sum

Python builtin_function_or_method: <built-in function sum>

In [38]:
b = @benchmark $pysum($x_py)

BenchmarkTools.Trial: 102 samples with 1 evaluation.
 Range (min … max):  48.138 ms … 53.446 ms  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     48.654 ms              ┊ GC (median):    0.00%
 Time  (mean ± σ):   49.382 ms ±  1.450 ms  ┊ GC (mean ± σ):  0.00% ± 0.00%

   ▅▂█ ▂                                                       
  ▇███▇█▇█▄▄▅▁▃▁▄▁▃▁▃▃▃▃▁▁▁▄▁▁▄▁▃▄▁▁▃▃▁▃▁▁▄▁▁▁▁▃▄▃▇▃▁▁▁▁▁▁▁▁▄ ▃
  48.1 ms         Histogram: frequency by time        53.2 ms <

 Memory estimate: 16 bytes, allocs estimate: 1.

In [39]:
d["Python (built-in)"] = minimum(b.times) / 1e6

48.138016

## C

### hand-written

In [40]:
c_code = """
#include <stddef.h>
double c_sum(size_t n, double *X) {
    double s = 0.0;
    for (size_t i = 0; i < n; ++i) {
        s += X[i];
    }
    return s;
}
""";

In [41]:
# compile to a shared library by piping C_code to gcc:
# (only works if you have gcc installed)
using Libdl
const Clib = tempname() * "." * Libdl.dlext

"/var/folders/1d/sbf5s24x6y306tgv_4hh06y00000gn/T/jl_xkroHIFk1I.dylib"

In [42]:
open(`gcc -fPIC -O3 -msse3 -xc -shared -o $Clib -`, "w") do f
    print(f, c_code)
end

In [43]:
c_sum(X::Array{Float64}) = @ccall Clib.c_sum(length(X)::Csize_t, X::Ptr{Float64})::Float64

c_sum (generic function with 1 method)

In [44]:
c_sum(x) ≈ sum(x)

true

In [45]:
b = @benchmark c_sum($x)

BenchmarkTools.Trial: 316 samples with 1 evaluation.
 Range (min … max):  15.574 ms …  22.223 ms  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     15.711 ms               ┊ GC (median):    0.00%
 Time  (mean ± σ):   15.824 ms ± 501.538 μs  ┊ GC (mean ± σ):  0.00% ± 0.00%

  █▆▂                                                           
  ███▆▆▅▇▄▄▆▅▅▆▄▄▄▃▂▂▃▃▂▂▁▂▁▂▂▁▁▁▂▁▂▃▂▁▂▁▁▂▂▁▁▁▁▁▂▁▁▁▁▁▁▁▁▁▁▁▃ ▃
  15.6 ms         Histogram: frequency by time         17.3 ms <

 Memory estimate: 0 bytes, allocs estimate: 0.

In [46]:
d["C"] = minimum(b.times) / 1e6

15.574109

### hand-written (with `-fast-math`)

In [47]:
const Clib_fastmath = tempname() * "." * Libdl.dlext

# The same as above but with a -ffast-math flag added
open(`gcc -fPIC -O3 -msse3 -xc -shared -ffast-math -o $Clib_fastmath -`, "w") do f
    print(f, c_code) 
end

# define a Julia function that calls the C function:
c_sum_fastmath(X::Array{Float64}) = @ccall Clib_fastmath.c_sum(length(X)::Csize_t, X::Ptr{Float64})::Float64

c_sum_fastmath (generic function with 1 method)

In [48]:
b = @benchmark c_sum_fastmath($x)

BenchmarkTools.Trial: 1012 samples with 1 evaluation.
 Range (min … max):  4.682 ms …   7.089 ms  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     4.847 ms               ┊ GC (median):    0.00%
 Time  (mean ± σ):   4.930 ms ± 325.111 μs  ┊ GC (mean ± σ):  0.00% ± 0.00%

  ▁▄█▅▃▅▅▁                                                     
  ████████▆▄▄▄▃▃▃▄▃▃▃▂▂▂▂▂▂▂▃▂▃▂▂▂▁▃▁▂▁▂▂▂▁▂▂▂▂▂▁▁▁▂▂▂▂▂▂▁▁▂▂ ▃
  4.68 ms         Histogram: frequency by time        6.55 ms <

 Memory estimate: 0 bytes, allocs estimate: 0.

In [49]:
d["C (fastmath)"] = minimum(b.times) / 1e6

4.681794

## Julia

### built-in

In [50]:
b = @benchmark sum($x)

BenchmarkTools.Trial: 997 samples with 1 evaluation.
 Range (min … max):  4.583 ms …   8.944 ms  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     4.909 ms               ┊ GC (median):    0.00%
 Time  (mean ± σ):   5.003 ms ± 364.629 μs  ┊ GC (mean ± σ):  0.00% ± 0.00%

       ▃▄▅▇█▆▆▆▂▁                                              
  ▃▃▄▄▇██████████▇▅▃▅▃▄▃▃▃▂▃▂▃▂▃▃▂▂▃▃▃▃▂▃▂▃▃▃▂▂▂▂▂▂▁▂▂▂▂▂▁▁▁▂ ▃
  4.58 ms         Histogram: frequency by time        6.39 ms <

 Memory estimate: 0 bytes, allocs estimate: 0.

In [51]:
d["Julia (built-in)"] = minimum(b.times) / 1e6

4.583328

### built-in (with `Vector{Any}`)

In [52]:
x_any = Vector{Any}(x)
b = @benchmark sum($x_any)

BenchmarkTools.Trial: 16 samples with 1 evaluation.
 Range (min … max):  288.235 ms … 329.311 ms  ┊ GC (min … max): 0.00% … 8.77%
 Time  (median):     315.663 ms               ┊ GC (median):    8.83%
 Time  (mean ± σ):   315.167 ms ±  10.099 ms  ┊ GC (mean ± σ):  7.86% ± 3.05%

                                        ▁█     ▁                 
  ▆▁▁▁▁▁▁▁▁▁▁▆▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁██▁▁▆▆▁█▁▁▁▆▁▆▁▁▆▁▁▁▁▁▆ ▁
  288 ms           Histogram: frequency by time          329 ms <

 Memory estimate: 152.59 MiB, allocs estimate: 9999999.

In [53]:
d["Julia (built-in, Any)"] = minimum(b.times) / 1e6

288.235337

### hand-written

In [54]:
function mysum(A)
    s = zero(eltype(A)) # the correct type of zero for A
    for a in A
        s += a
    end
    return s
end

mysum (generic function with 1 method)

In [55]:
b = @benchmark mysum($x)

BenchmarkTools.Trial: 316 samples with 1 evaluation.
 Range (min … max):  15.811 ms … 16.011 ms  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     15.833 ms              ┊ GC (median):    0.00%
 Time  (mean ± σ):   15.841 ms ± 26.779 μs  ┊ GC (mean ± σ):  0.00% ± 0.00%

      █▅▄▄   ▂                                                 
  ▄█▆█████▅▆▇█▅▆▇▅▄▆▆▅▃▂▄▄▅▂▄▂▄▃▃▄▂▃▅▄▄▅▃▃▁▂▂▂▃▂▁▁▁▁▁▁▁▁▂▁▁▁▃ ▃
  15.8 ms         Histogram: frequency by time        15.9 ms <

 Memory estimate: 0 bytes, allocs estimate: 0.

In [56]:
d["Julia (hand-written)"] = minimum(b.times) / 1e6

15.810737

### hand-written (with `@fastmath`)

In [57]:
function mysum_fastmath(A)
    s = zero(eltype(A)) # the correct type of zero for A
    @fastmath for a in A
        s += a
    end
    return s
end

mysum_fastmath (generic function with 1 method)

In [58]:
b = @benchmark mysum_fastmath($x)

BenchmarkTools.Trial: 1043 samples with 1 evaluation.
 Range (min … max):  4.463 ms …   5.678 ms  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     4.763 ms               ┊ GC (median):    0.00%
 Time  (mean ± σ):   4.786 ms ± 121.664 μs  ┊ GC (mean ± σ):  0.00% ± 0.00%

                  ▂ ▃▃█▆▂                                      
  ▂▂▂▂▂▃▂▂▂▃▃▃▂▂▃▅███████▇▅▄▅▅▄▄▄▃▄▃▃▃▃▂▂▂▂▂▂▂▂▂▂▂▂▁▁▁▂▁▂▁▂▁▂ ▃
  4.46 ms         Histogram: frequency by time        5.31 ms <

 Memory estimate: 0 bytes, allocs estimate: 0.

In [59]:
d["Julia (hand-written, fastmath)"] = minimum(b.times) / 1e6

4.462856

## Summary

In [60]:
for (key, value) in sort(collect(d), by=x->x[2])
    println(rpad(key, 30, "."), lpad(round(value, digits=2), 10, "."))
end

Julia (hand-written, fastmath)......4.46
Julia (built-in)....................4.58
C (fastmath)........................4.68
Python (numpy)......................6.08
C..................................15.57
Julia (hand-written)...............15.81
Python (built-in)..................48.14
Julia (built-in, Any).............288.24
Python (hand-written).............340.15


And of course, our hand-written Julia implementation is type-generic!

In [61]:
mysum_fastmath(rand(ComplexF64, 10))

5.517806346353696 + 4.858273664496539im

# Supplement: What about other functions?

### Log

In [ ]:
@which log(1.2)

In [ ]:
using BenchmarkTools

# uses the system C library
clog(x) = ccall(:log, Float64, (Float64,), x)
# uses LLVM's log
llvmlog(x) =  ccall(Symbol("llvm.log.f64"), llvmcall, Float64, (Float64,), x)

@btime log(1.2)    
@btime clog(1.2)    
@btime llvmlog(1.2);

### Exp

In [ ]:
@which exp(1.2)

In [ ]:
using BenchmarkTools

# uses the system C library
cexp(x) = ccall(:exp, Float64, (Float64,), x)
# uses LLVM's
llvmexp(x) =  ccall(Symbol("llvm.exp.f64"), llvmcall, Float64, (Float64,), x)

@btime exp(1.2);
@btime cexp(1.2);
@btime llvmexp(1.2);

### Matrix multiplication

In [ ]:
N = 10
C = zeros(N,N);
A = rand(N,N);
B = rand(N,N);

In [ ]:
using LinearAlgebra

mul!(C, A, B); # "built-in", calls underlying BLAS/LAPACK

In [ ]:
C ≈ A * B

In [ ]:
using BenchmarkTools

function mul_naive!(C, A, B)
    for m in axes(A,1)
        for n in axes(B,2)
            Cmn = zero(eltype(C))
            for k in axes(A,2)
               @inbounds Cmn += A[m,k] * B[k,n]
            end
            C[m,n] = Cmn
        end
   end
end

In [ ]:
mul_naive!(C,A,B)
C ≈ A * B

[LoopVectorization.jl](https://github.com/chriselrod/LoopVectorization.jl)

In [ ]:
using LoopVectorization

function mul_turbo!(C, A, B)
    @turbo for m in axes(A,1)
        for n in axes(B,2)
            Cmn = zero(eltype(C))
            for k in axes(A,2)
               @inbounds Cmn += A[m,k] * B[k,n]
            end
            C[m,n] = Cmn
        end
   end
end

In [ ]:
mul_turbo!(C,A,B)
C ≈ A * B

In [ ]:
c_code = """
#include <stddef.h>
#include <math.h>

void gemm(double* restrict C, double* restrict A, double* restrict B, long M, long K, long N){
  for (long i = 0; i < M*N; i++){
    C[i] = 0.0;
  }
  for (long n = 0; n < N; n++){
    for (long k = 0; k < K; k++){
      for (long m = 0; m < M; m++){
	C[m + n*M] += A[m + k*M] * B[k + n*K];
      }
    }
  }
  return;
}
""";

In [ ]:
# compile to a shared library by piping C_code to gcc:
# (only works if you have gcc installed)
using Libdl
const Clib_gemm = tempname() * "." * Libdl.dlext

open(`gcc -fPIC -O3 -msse3 -xc -shared -o $Clib_gemm -`, "w") do f
    print(f, c_code)
end

c_gemm(C::Array{Float64},A::Array{Float64},B::Array{Float64}) = @ccall Clib_gemm.gemm(C::Ptr{Float64}, A::Ptr{Float64}, B::Ptr{Float64}, size(A,1)::Clong, size(A,2)::Clong, size(B,2)::Clong)::Cvoid

In [ ]:
c_gemm(C, A, B)
C ≈ A*B

In [ ]:
@btime mul_naive!($C,$A,$B);
@btime mul_turbo!($C,$A,$B);
@btime mul!($C, $A, $B);
@btime c_gemm($C, $A, $B)

**Note for larger `N`:** BLAS is multithreaded for larger `N`. In this case our `mul_avx!` can be slower than `mul!`.